# Evaluation

Preparation:

In [1]:
from problem_description_and_state import ProblemDescription, State
from data_exploration_classes import Data
from training_classes import TransitionProbabilityLearner, TransitionProbabilityMatrix, RewardMatrixS1, RewardMatrixSA
from evaluation_classes import GreedyPolicy, PolicyFromMDPSolver, Evaluation
import collections
import datetime
import mdptoolbox
import numpy

Problem = collections.namedtuple('Problem', ['pd', 'data', 'learner', 'tpms', 'rm', 'greedy_scores', 'results'])
Result = collections.namedtuple('Result', [
    'solver', 'discount_factor', 'max_iter', 'time_real', 'time_solver', 'policy', 'scores'])

def create_problem(cols, rows, datafiles):
    pd = ProblemDescription(cols, rows)
    data = [Data(pd, datafile) for datafile in datafiles]
    # learn transition probabilities from the training data
    learner = TransitionProbabilityLearner(pd, data[0], min_support=5)
    # create the transition probability matrices
    tpms = list(TransitionProbabilityMatrix(pd, learner, dtype=numpy.float64))
    # tried other dtypes as well, but float32 and float16 produced errors in mdptoolbox
    # (incompatible datatypes for float16, t.p.m. not stochastic for float32)
    rm = RewardMatrixSA(pd, dtype=numpy.int16).get()
    # SA stands for the shape of the reward matrix, i.e. one matrix with shape SxA
    # instead of a list of multiple matrices of shape Sx1 for each action (which didn't work with mdptoolbox)
    return Problem(pd, data, learner, tpms, rm, [], [])

p4 = create_problem(2, 2, ['data/2x2/warehouse{}.txt'.format(s) for s in ['training', 'order', 'ordernew']])
p6 = create_problem(3, 2, ['data/3x2/warehouse{}.txt'.format(s) for s in ['training', 'order']])

## Establishing a Performance Reference through a Greedy Algorithm

A greedy algorithm always uses the closest applicable slot to satisfy a request. It has no notion of planning and does not care for the distribution of item colors.

Comparing its sum of rewards over a series of requests given as training data or test data can serve to establish a reference value to compare optimal policies as solutions from a MDP to. Instead of the sum of rewards derived values like average reward per request or normalized score values can be used as well.

One such score is defined as follows:
* compute the average reward per request (= sum of rewards / #requests)
  * note: rewards are integers in the range of (-infty, 0]
* divide that by the minimum (i.e. worst) reward under normal circumstances (i.e. without invalid states, invalid requests or invalid actions)
* multiply by -1 to get a negative number again
* add 1
* replace any remaining negative values (caused by invalid states etc.) by 0.0
* the resulting score is now in the range \[0.0, 1.0\]
* 0.0 is the worst achievable performance under normal circumstances, 1.0 is perfect performance (only achievable given optimal requests, i.e. only need to use the last inventory slot for all requests)

### Testing the Greedy Algorithm

In [2]:
pd = p6.pd  # use the 3x2 inventory
gpol = GreedyPolicy(pd)  
verb_store = 0
verb_restore = 1
color1 = 0
last_slot = pd.number_of_inventory_slots - 1
# inventory slot numbers:
# 0  1  2
# 3  4  5 <-+
#       ^---+-- last slot, highest reward (=0)
assert gpol.get_action(State([0,0,0,0,0,0], verb_store, color1).get_index(pd)) == last_slot
assert gpol.get_action(State([0,0,0,0,1,1], verb_store, color1).get_index(pd)) == 2
assert gpol.get_action(State([0,1,1,0,1,1], verb_store, color1).get_index(pd)) == 3
assert gpol.get_action(State([0,1,1,1,1,1], verb_store, color1).get_index(pd)) == 0

assert gpol.get_action(State([1,1,1,1,1,1], verb_restore, color1).get_index(pd)) == last_slot
assert gpol.get_action(State([1,1,1,1,0,0], verb_restore, color1).get_index(pd)) == 2
assert gpol.get_action(State([1,0,0,1,0,0], verb_restore, color1).get_index(pd)) == 3
assert gpol.get_action(State([1,0,0,0,0,0], verb_restore, color1).get_index(pd)) == 0

assert gpol.get_action(State([2,2,2,1,1,1], verb_restore, color1 + 1).get_index(pd)) == 2
assert gpol.get_action(State([1,1,1,2,0,0], verb_restore, color1 + 1).get_index(pd)) == 3

ignore_request_action = pd.number_of_actions - 1
assert gpol.get_action(State([1,1,1,1,1,1], verb_store, color1).get_index(pd)) == ignore_request_action
assert gpol.get_action(State([2,2,2,2,2,2], verb_store, color1).get_index(pd)) == ignore_request_action
assert gpol.get_action(State([2,2,2,2,2,2], verb_restore, color1).get_index(pd)) == ignore_request_action
assert gpol.get_action(State([0,0,0,0,0,0], verb_restore, color1).get_index(pd)) == ignore_request_action

### Evaluating the Greedy Algorithm

In [3]:
for problem in [p6, p4]:
    gpol = GreedyPolicy(problem.pd)
    problem.greedy_scores.clear()
    for data in problem.data:
        eva = Evaluation(problem.pd, data, gpol)
        problem.greedy_scores.append(eva.get_score())
        print('using data from {} with {} requests'.format(data.filepath, len(data.requests)))
        print('- total reward {}'.format(eva.get_total_reward()))
        print('- average reward per request {:.3f}'.format(eva.get_total_reward() / len(data.requests)))
        print('- minimum achievable reward per request {:.3f}'.format(min((
                -problem.pd.get_manhattan_distance_to_last_inventory_slot(slot)
                for slot in range(problem.pd.number_of_inventory_slots)
        ))))
        print('- score {:.3f}'.format(eva.get_score()))
        print()

using data from data/3x2/warehousetraining.txt with 12108 requests
- total reward -13578
- average reward per request -1.121
- minimum achievable reward per request -3.000
- score 0.626

using data from data/3x2/warehouseorder.txt with 60 requests
- total reward -64
- average reward per request -1.067
- minimum achievable reward per request -3.000
- score 0.644

using data from data/2x2/warehousetraining.txt with 8177 requests
- total reward -6224
- average reward per request -0.761
- minimum achievable reward per request -2.000
- score 0.619

using data from data/2x2/warehouseorder.txt with 65 requests
- total reward -49
- average reward per request -0.754
- minimum achievable reward per request -2.000
- score 0.623

using data from data/2x2/warehouseordernew.txt with 20 requests
- total reward -19
- average reward per request -0.950
- minimum achievable reward per request -2.000
- score 0.525



## Comparing the Greedy Algorithm with an Optimal Policy as Computed by a MDP Solver

In [4]:
# reset solver results from previous runs
p4.results.clear()
p6.results.clear()

In [5]:
def evaluate_solver(problem, solver_class, discount_factor, iterations, show_training_score_only=True):
    start = datetime.datetime.now()
    solver = solver_class(
        problem.tpms,
        problem.rm,
        discount_factor,
        max_iter=iterations
    )
    try:
        solver.run()
    except MemoryError:
        print('{}x{}: {}(discount_factor={}, max_iter={}) could not run due to memory issues'.format(
            problem.pd.number_of_inventory_cols, problem.pd.number_of_inventory_rows,
            type(solver).__name__, discount_factor, iterations
        ))
        return
    stop = datetime.datetime.now()
    time_real = (stop - start).total_seconds()
    time_solver = solver.time
    policy = PolicyFromMDPSolver(problem.pd, solver)
    result = Result(type(solver).__name__, discount_factor, iterations, time_real, time_solver, policy, [])
    print('{}x{}: {}(discount_factor={}, max_iter={}) ran for {:.1f}s ({:.1f}s including initialization)'.format(
        problem.pd.number_of_inventory_cols, problem.pd.number_of_inventory_rows,
        result.solver, result.discount_factor, result.max_iter, result.time_solver, result.time_real
    ))
    max_filenamelength = max((len(data.filepath) for data in problem.data))
    for data_index, data in enumerate(problem.data):
        score = Evaluation(problem.pd, data, policy).get_score()
        result.scores.append(score)
        if show_training_score_only and data_index > 0:
            continue
        print(('\t{: <' + str(max_filenamelength) + '}: solver score {:.3f} vs greedy score {:.3f}').format(
            data.filepath, score, problem.greedy_scores[data_index]
        ))

In [18]:
evaluate_solver(p4, mdptoolbox.mdp.PolicyIteration, 0.95, 10, False)
evaluate_solver(p6, mdptoolbox.mdp.PolicyIteration, 0.95, 10, False)
print('-' * 80)
evaluate_solver(p4, mdptoolbox.mdp.PolicyIterationModified, 0.95, 10, False)
evaluate_solver(p6, mdptoolbox.mdp.PolicyIterationModified, 0.95, 10, False)
print('-' * 80)
evaluate_solver(p4, mdptoolbox.mdp.ValueIteration, 0.95, 5000, False)
evaluate_solver(p6, mdptoolbox.mdp.ValueIteration, 0.95, 5000, False)
print('-' * 80)
evaluate_solver(p4, mdptoolbox.mdp.ValueIterationGS, 0.95, 5000, False)
# evaluate_solver(p6, mdptoolbox.mdp.ValueIterationGS, 0.95, 5000, False)
print('3x2: ValueIterationGS skipped due to excessive runtime')
print('-' * 80)
evaluate_solver(p4, mdptoolbox.mdp.RelativeValueIteration, 0.95, 5000, False)
evaluate_solver(p6, mdptoolbox.mdp.RelativeValueIteration, 0.95, 5000, False)

2x2: PolicyIteration(discount_factor=0.95, max_iter=10) ran for 2.3s (2.5s including initialization)
	data/2x2/warehousetraining.txt: solver score 0.619 vs greedy score 0.619
	data/2x2/warehouseorder.txt   : solver score 0.623 vs greedy score 0.623
	data/2x2/warehouseordernew.txt: solver score 0.525 vs greedy score 0.525
3x2: PolicyIteration(discount_factor=0.95, max_iter=10) could not run due to memory issues
--------------------------------------------------------------------------------
2x2: PolicyIterationModified(discount_factor=0.95, max_iter=10) ran for 0.8s (1.0s including initialization)
	data/2x2/warehousetraining.txt: solver score 0.619 vs greedy score 0.619
	data/2x2/warehouseorder.txt   : solver score 0.623 vs greedy score 0.623
	data/2x2/warehouseordernew.txt: solver score 0.525 vs greedy score 0.525
3x2: PolicyIterationModified(discount_factor=0.95, max_iter=10) ran for 184.6s (243.2s including initialization)
	data/3x2/warehousetraining.txt: solver score 0.626 vs greedy

### Results

All MDP algorithms produce policies which perform exactly as good as a greedy policy when executed on the request sequences of any of the provided data files. Given that the parameters like discount factor and number of iterations are already quite reasonable default values for these algorithms the implication is that there simply is no better policy for such request sequences.

This further implies that it is not beneficial for the agent to plan ahead for the patterns discovered in the data exploration notebook. Reverse engineering the generator function that produced the data files showed (disregarding the sequential request patterns) that the probability of a restore request is roughly (#occupied slots) / (#inventory slots), and that the probability of the color of the item to be restored is roughly (#items of that color in the inventory) / (#occupied slots). This explains why a greedy policy is already optimal. Even though red items are twice as likely to appear as blue or white, their retrieval frequency seems to mostly depend on their color ratio in the inventory. In other words, storing a blue item further away from the starting position than a red item makes no sense, because all items in the inventory are equally likely to be retrieved.

There is further evidence to back this up: tweaking the `min_support` parameter of the TransitionProbabilityLearner class by raising it from its default value of 5 to, for example, `1e10` forces the class for practically all state transitions to fall back to the least granular data available (totally disregarding the current inventory content) and to _only use the color distribution_ among all requests to determine the probability of state transitions (verbs are assumed to be equally distributed). Under this condition the agent actually plans ahead _worse_ when the discount factor is _increased_, because its decisions are based on bad data. A quick demonstration with the 2x2 problem and the PolicyIteration solver:

In [21]:
learner_mod = TransitionProbabilityLearner(p4.pd, p4.data[0], min_support=1e10)
tpms_mod = list(TransitionProbabilityMatrix(p4.pd, learner_mod, dtype=numpy.float64))
p4_mod = Problem(p4.pd, p4.data, learner_mod, tpms_mod, p4.rm, p4.greedy_scores, [])
for discount_factor in [0.2, 0.6, 0.8, 0.9, 0.95, 0.99, 0.999]:
    evaluate_solver(p4_mod, mdptoolbox.mdp.PolicyIteration, discount_factor, 10, True)

2x2: PolicyIteration(discount_factor=0.2, max_iter=10) ran for 2.2s (2.4s including initialization)
	data/2x2/warehousetraining.txt: solver score 0.619 vs greedy score 0.619
2x2: PolicyIteration(discount_factor=0.6, max_iter=10) ran for 2.2s (2.4s including initialization)
	data/2x2/warehousetraining.txt: solver score 0.619 vs greedy score 0.619
2x2: PolicyIteration(discount_factor=0.8, max_iter=10) ran for 2.2s (2.4s including initialization)
	data/2x2/warehousetraining.txt: solver score 0.618 vs greedy score 0.619
2x2: PolicyIteration(discount_factor=0.9, max_iter=10) ran for 2.2s (2.3s including initialization)
	data/2x2/warehousetraining.txt: solver score 0.616 vs greedy score 0.619
2x2: PolicyIteration(discount_factor=0.95, max_iter=10) ran for 2.2s (2.4s including initialization)
	data/2x2/warehousetraining.txt: solver score 0.613 vs greedy score 0.619
2x2: PolicyIteration(discount_factor=0.99, max_iter=10) ran for 2.2s (2.4s including initialization)
	data/2x2/warehousetraining.

## Further Discussion of the MDP Algorithms Based on Generated Data

The existing data is not suitable to demonstrate the influence of parameter choice on convergence of the algorithms. Data generated with different characteristics can change that.

The characteristics of the requests generated by the ModifiedRequestGenerator class differ from those of the existing data in that red is 4 times as likely -- even for restore requests -- and the color probability of restore requests does not depend on the number of items of that color in the inventory (except that at least one item of that color needs to exist). It is also missing any sequential patterns.

### Preparation

In [6]:
from data_exploration_classes import ModifiedRequestGenerator, GeneratedData

data_gen = GeneratedData(p4.pd)
data_gen.requests = ModifiedRequestGenerator(p4.pd, data_gen.limit)
learner_gen = TransitionProbabilityLearner(p4.pd, data_gen, min_support=5)
tpms_gen = list(TransitionProbabilityMatrix(p4.pd, learner_gen, dtype=numpy.float64))
gpol_gen = GreedyPolicy(p4.pd)
gpol_gen_eval = Evaluation(p4.pd, data_gen, gpol_gen)
p4_gen = Problem(p4.pd, [data_gen], learner_gen, tpms_gen, p4.rm, [gpol_gen_eval.get_score()], [])

### Proof and Influence of Parameter Choice

#### 1. PolicyIteration

In [8]:
for max_iter in [1, 2, 4, 8]:
    evaluate_solver(p4_gen, mdptoolbox.mdp.PolicyIteration, 0.95, max_iter)
print('-' * 80)
for discount_factor in [0.6, 0.8, 0.9, 0.95, 0.99, 0.999]:
    evaluate_solver(p4_gen, mdptoolbox.mdp.PolicyIteration, discount_factor, 10)

2x2: PolicyIteration(discount_factor=0.95, max_iter=1) ran for 0.2s (0.4s including initialization)
	generated data: solver score 0.701 vs greedy score 0.701
2x2: PolicyIteration(discount_factor=0.95, max_iter=2) ran for 0.4s (0.6s including initialization)
	generated data: solver score 0.709 vs greedy score 0.701
2x2: PolicyIteration(discount_factor=0.95, max_iter=4) ran for 0.9s (1.1s including initialization)
	generated data: solver score 0.709 vs greedy score 0.701
2x2: PolicyIteration(discount_factor=0.95, max_iter=8) ran for 1.8s (1.9s including initialization)
	generated data: solver score 0.709 vs greedy score 0.701
--------------------------------------------------------------------------------
2x2: PolicyIteration(discount_factor=0.6, max_iter=10) ran for 2.3s (2.5s including initialization)
	generated data: solver score 0.701 vs greedy score 0.701
2x2: PolicyIteration(discount_factor=0.8, max_iter=10) ran for 2.4s (2.5s including initialization)
	generated data: solver score

This shows that PolicyIteration does not need more than 10 iterations. It also demonstrates that a discount factor of 0.95 is still a bit low to generate a policy that fully exploits the benefits of long-term planning.

#### 2. PolicyIterationModified

In [14]:
for max_iter in [1, 2, 4, 8, 16, 32]:
    evaluate_solver(p4_gen, mdptoolbox.mdp.PolicyIterationModified, 0.95, max_iter)
print('-' * 80)
for discount_factor in [0.6, 0.8, 0.9, 0.95, 0.99, 0.999]:
    evaluate_solver(p4_gen, mdptoolbox.mdp.PolicyIterationModified, discount_factor, 10)

2x2: PolicyIterationModified(discount_factor=0.95, max_iter=1) ran for 3.1s (3.2s including initialization)
	generated data: solver score 0.709 vs greedy score 0.701
2x2: PolicyIterationModified(discount_factor=0.95, max_iter=2) ran for 2.1s (2.3s including initialization)
	generated data: solver score 0.709 vs greedy score 0.701
2x2: PolicyIterationModified(discount_factor=0.95, max_iter=4) ran for 1.3s (1.5s including initialization)
	generated data: solver score 0.709 vs greedy score 0.701
2x2: PolicyIterationModified(discount_factor=0.95, max_iter=8) ran for 0.9s (1.1s including initialization)
	generated data: solver score 0.709 vs greedy score 0.701
2x2: PolicyIterationModified(discount_factor=0.95, max_iter=16) ran for 0.6s (0.8s including initialization)
	generated data: solver score 0.709 vs greedy score 0.701
2x2: PolicyIterationModified(discount_factor=0.95, max_iter=32) ran for 0.4s (0.6s including initialization)
	generated data: solver score 0.709 vs greedy score 0.701
--

PolicyIterationModified converges even faster than PolicyIteration. However, it is very sensitive to a high discount factor, i.e. the computation time increases drastically. Curiously, the runtime for very low iterations (only 1 or 2) is higher than for higher iterations (like around 10).

#### 3. ValueIteration

In [12]:
for max_iter in [1, 100, 10000]:
    evaluate_solver(p4_gen, mdptoolbox.mdp.ValueIteration, 0.95, max_iter)
print('-' * 80)
for discount_factor in [0.6, 0.8, 0.9, 0.95, 0.99, 0.999]:
    evaluate_solver(p4_gen, mdptoolbox.mdp.ValueIteration, discount_factor, 100)

2x2: ValueIteration(discount_factor=0.95, max_iter=1) ran for 0.0s (2.5s including initialization)
	generated data: solver score 0.709 vs greedy score 0.701
2x2: ValueIteration(discount_factor=0.95, max_iter=100) ran for 0.0s (2.4s including initialization)
	generated data: solver score 0.709 vs greedy score 0.701
2x2: ValueIteration(discount_factor=0.95, max_iter=10000) ran for 0.0s (2.4s including initialization)
	generated data: solver score 0.709 vs greedy score 0.701
--------------------------------------------------------------------------------
2x2: ValueIteration(discount_factor=0.6, max_iter=100) ran for 0.0s (2.4s including initialization)
	generated data: solver score 0.701 vs greedy score 0.701
2x2: ValueIteration(discount_factor=0.8, max_iter=100) ran for 0.0s (2.3s including initialization)
	generated data: solver score 0.701 vs greedy score 0.701
2x2: ValueIteration(discount_factor=0.9, max_iter=100) ran for 0.0s (2.4s including initialization)
	generated data: solver sc

This shows that ValueIteration probably ignores the `max_iter` parameter. The result for the discount factor is the same as in PolicyIteration. There is a slight sensitivity to a high discount factor in terms of computation time.

#### 4. ValueIterationGS

In [ ]:
for max_iter in [1, 10, 100, 1000]:
    evaluate_solver(p4_gen, mdptoolbox.mdp.ValueIterationGS, 0.95, max_iter)
print('-' * 80)
for discount_factor in [0.6, 0.8, 0.9, 0.95, 0.99, 0.999]:
    evaluate_solver(p4_gen, mdptoolbox.mdp.ValueIterationGS, discount_factor, 100)

2x2: ValueIterationGS(discount_factor=0.95, max_iter=1) ran for 144.5s (146.9s including initialization)
	generated data: solver score 0.709 vs greedy score 0.701
2x2: ValueIterationGS(discount_factor=0.95, max_iter=10) ran for 144.8s (147.2s including initialization)
	generated data: solver score 0.709 vs greedy score 0.701
2x2: ValueIterationGS(discount_factor=0.95, max_iter=100) ran for 141.3s (143.6s including initialization)
	generated data: solver score 0.709 vs greedy score 0.701
2x2: ValueIterationGS(discount_factor=0.95, max_iter=1000) ran for 144.3s (146.7s including initialization)
	generated data: solver score 0.709 vs greedy score 0.701
--------------------------------------------------------------------------------
2x2: ValueIterationGS(discount_factor=0.6, max_iter=100) ran for 11.5s (13.9s including initialization)
	generated data: solver score 0.701 vs greedy score 0.701
2x2: ValueIterationGS(discount_factor=0.8, max_iter=100) ran for 28.5s (30.8s including initializat

ValueIterationGS seems to ignore the max_iter parameter. It is quite sensitive to a high discount factor, resulting in long computation times compared to other algorithms.

#### 5. RelativeValueIteration

In [15]:
for max_iter in [1, 100, 10000]:
    evaluate_solver(p4_gen, mdptoolbox.mdp.RelativeValueIteration, 0.95, max_iter)
print('-' * 80)
for discount_factor in [0.6, 0.8, 0.9, 0.95, 0.99, 0.999]:
    evaluate_solver(p4_gen, mdptoolbox.mdp.RelativeValueIteration, discount_factor, 100)

2x2: RelativeValueIteration(discount_factor=0.95, max_iter=1) ran for 0.0s (0.2s including initialization)
	generated data: solver score 0.701 vs greedy score 0.701
2x2: RelativeValueIteration(discount_factor=0.95, max_iter=100) ran for 0.0s (0.2s including initialization)
	generated data: solver score 0.710 vs greedy score 0.701
2x2: RelativeValueIteration(discount_factor=0.95, max_iter=10000) ran for 2.3s (2.5s including initialization)
	generated data: solver score 0.710 vs greedy score 0.701
--------------------------------------------------------------------------------
2x2: RelativeValueIteration(discount_factor=0.6, max_iter=100) ran for 0.0s (0.2s including initialization)
	generated data: solver score 0.710 vs greedy score 0.701
2x2: RelativeValueIteration(discount_factor=0.8, max_iter=100) ran for 0.0s (0.2s including initialization)
	generated data: solver score 0.710 vs greedy score 0.701
2x2: RelativeValueIteration(discount_factor=0.9, max_iter=100) ran for 0.0s (0.3s incl

RelativeValueIteration is much faster than ValueIteration but converges slower. Even a high discount factor does not seem to have any impact on policy or computation time.